# Band Function

This program shows you how to calcualte the integrated photon and energy fluxes of the Band Function using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Band Function can be found in the following paper:
### Band, D., Matteson, J., Ford, L., et al. 1993, ApJ, 413, 281  Equation 1.
http://adsabs.harvard.edu/abs/1993ApJ...413..281B


Original Function, before integration: 

<br />
<br />

for   $E < (\ (\alpha - \beta) \times E_0 \ )$
<br />
<br />
\begin{equation}
f\left(E\right) = N \times \left(\frac{E}{100}\right)^{\alpha} \times  \exp \left( \frac{-E}{E_0} \right) dE \ \ \ \ 
\end{equation}
<br />
<br />
for   $E >= (\ (\alpha - \beta) \times E_0 \ )$
<br />
<br />
\begin{equation}
f\left(E\right) = N \times \left(\frac{(\alpha-\beta) \times E_0}{100}\right)^{(\alpha - \beta)} \left(\frac{E}{100}\right)^{\beta} \times \exp \left(-(\alpha - \beta) \right) dE \ \ \ \ 
\end{equation}

### My version of this equation:

We use E-folding energy instead of Epeak here, but Epeak can easily be derived by using the equation in the next block.

t is used to represent the e-folding energy because in PYXSPEC, the parameter name for it is 'tem'.  
    
    alpha: low energy index
    beta:  high energy index
    a - alpha
    b - beta
    t:     e-folding energy (E_0, or tem)
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV
        

    for E < ((a-b) * t):
        f(E)= N * (((E/100.0)**a) * exp(-E/t)) dE
        
    for E >= ((a-b) * t):
        f(E)= N * ((((a-b)*t)/100.0)**(a-b)*((E/100)**b)*exp(-(a-b))) dE


### Converstion between E-folding energy and Epeak energy:
Sometimes you'll see the energy as Epeak.

    epk    = 516.59979
    alpha  = -1.0355458
    efold  = (epk)/(alpha + 2.)
    efold  = (516.59979)/(-1.0355458 + 2.)

\begin{equation}
E_{pk} = E_0 \times \left( \alpha + 2.0 \right) \ \ \ \ \ 
\end{equation}

# Begin Program

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants

In [2]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

### Parameters
    pars = [alpha, beta, ebreak, norm]

In [3]:
pars     = [-1.0355458, -2.2196138, 535.6395254435099, 0.017525647]  

# normalization
norm   = pars[-1]

def get_parVals():
    pars     = [-1.0355458, -2.2196138, 535.6395254435099, 0.017525647]  
    return pars


# FUNCTION INTEGRATED

Make a flux array of 5000 elements that will be filled with flux increments.
An energy array with 5000 energies ranging from 10 to 10,000 keV in log space will be passed to the integrated BAND function and will calculate an increment of flux for each energy element passed to it.  
Those increments will be stored in the flux array of 5000 elements.

After running band(engs, flux, *pars) the fluxes (un-normalized) will be appended to the flux array.  Then you can multiply each flux element by the normalization factor.  
You could pass the normalization to the band function and multiply it within the function instead of at the end.  
I do not do that here because the equation (shown below) was the exact one  used during PYXSPEC fitting.  The only difference between this function and the PYXSPEC one is that the order the variables passed to the band function in PYXSPEC are:  band(engs, params, flux)


If you sum up the 5000 increments in the flux array and then multiply by the the normalization, you are getting the integrated flux from 10 to 10,000 keV.
You are essentially summing up the flux area under the flux curve.


In this example, we leave the BAND normalization parameter out of the function and multiply it at the end.  We do this because PYXSPEC uses this exact function for band (as below) during the Maximum Likelihood fitting process of parameter estimation.  

This is how PYXSPEC estiamtes its flux calculations when you use the commands:

    AllModels.setEnergies("10. 10000.")
    AllModels.calcFlux("10. 10000.0 err")
 
This is a crude method to estimate the flux and is not the  most accurate way.  The best way to calculate the integrated flux is to take the original function and numerically integrate it with scipy.integrate.quad for quadrature integration.

    Photon Flux Units:  photons s^-1 cm^-2
    Energy Flux Units:  ergs s^-1 cm^-2




In [4]:
def band(engs, flux, *params):
    from numpy import exp
    from mpmath import gammainc
    from mpmath import fp   # use fp.gammainc, speeds up calculation
    import time
    
    a   = float(params[0])  # alpha
    b   = float(params[1])  # beta
    t   = float(params[2])  # e-folding energy, E_0 or tem
    
    start_time = time.time()

    for i in range(len(engs)-1):
        if engs[i] < ((a - b) * t):
            multiplier    = ((100.**(-a)) * (t**(a+1.)))
            lowIntegral   = float( fp.gammainc(a + 1., (engs[i]/t)) )
            highIntegral  = float( fp.gammainc(a + 1., (engs[i+1]/t)) )
            val           = multiplier * (lowIntegral - highIntegral)
            flux[i]       = val
        else:
            multiplier    = ((1./(b+1.))*((100.**(-a))*((a-b)**(a-b))*-exp(b-a)*(t**(a-b))))
            lowIntegral   = engs[i]**(b+1.)
            highIntegral  = engs[i+1]**(b+1.)
            val           = multiplier * (lowIntegral - highIntegral)
            flux[i]       = val
    stop_time = time.time() - start_time 
    print('time: %f seconds'%(stop_time))

#### We ran this function twice:
Once with fp.gammainc and once without it, gammainc.

fp stands for fast low-precision arithmetic.  This speeds up the process.

    With fp:     2.11708402634 seconds  
        Fluxes: 6.218796647 and 2.047850400e-06 
        
    Without fp:  6.34235787392 seconds
        Fluxes: 6.218796647 and 2.047850400e-06
        
You can see that the lower precision doesn't change the flux values.
http://docs.sympy.org/0.6.7/modules/mpmath/basics.html

In [5]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
band(engs, flux, *pars)
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN EBAND FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))

time: 1.810383 seconds

Photon Flux:  6.218796647 	 photons s^-1 cm^-2
Energy Flux:  2.047850400e-06 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.

In [6]:
def band(energy):
    from numpy import exp
    alpha, beta, tem, norm = get_parVals()
    
    
    a = float(alpha)
    b = float(beta)
    t = float(tem)
    N = float(norm)
    eng  = energy
    
    if eng < ( (a-b) * t ):
        return  N * (((eng/100.0)**a) * (exp(-eng/t)))
    else:
        return  N * (((((a-b)*t)/100.0)**(a-b)) * (exp(b-a))*((eng/100.0)**b))

def eband(energy):
    eng  = energy
    return eng * band(eng)


In [7]:
Flux_Ph = integrate.quad(band, emin, emax, limit=100)[0]
Flux_En = integrate.quad(eband, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  6.218796647 	 photons s^-1 cm^-2
Energy Flux:  2.049265654e-06 	 ergs s^-1 cm^-2

